In [2]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
%%bash 
pwd

/content/Template_Project


In [7]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)
from core.dl_framework.utils import get_specific_history
from core.dl_framework.utils import plot_history

In [10]:
from core.dl_framework.learner import Learner
from core.dl_framework.callbacks import Recorder, Monitor, EarlyStopping, Checkpoints
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
from torchvision import transforms
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"], CNN=True)

train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, test_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)
learn = Learner(train_ds, setup_config)
learn.fit(20)


100%|██████████| 187/187 [00:00<00:00, 1110.93it/s]


epoch: 1/20	[valid_acc: 0.9033	valid_loss: 0.6543	train_loss: 1.2569]
new best: valid_loss 0.6542993624580098
new checkpoint


100%|██████████| 187/187 [00:00<00:00, 1153.57it/s]


epoch: 2/20	[valid_acc: 0.9370	valid_loss: 0.3650	train_loss: 0.4757]
new best: valid_loss 0.3649834500436477
new checkpoint


100%|██████████| 187/187 [00:00<00:00, 1177.37it/s]


...

epoch: 18/20	[valid_acc: 0.9845	valid_loss: 0.0605	train_loss: 0.0507]
new best: valid_loss 0.0605442564695436
new checkpoint


100%|██████████| 187/187 [00:00<00:00, 1070.48it/s]


epoch: 19/20	[valid_acc: 0.9845	valid_loss: 0.0585	train_loss: 0.0479]
new best: valid_loss 0.058519634253838486
new checkpoint


100%|██████████| 187/187 [00:00<00:00, 1171.07it/s]

epoch: 20/20	[valid_acc: 0.9846	valid_loss: 0.0567	train_loss: 0.0455]
new best: valid_loss 0.05669835654092345
new checkpoint


In [5]:
from pathlib import Path
from datetime import datetime
ckp_path = "/content/gdrive/MyDrive/Marcel_Moczarski/Data_Science/Google_Services/Local_Drive_Storage/Conda_Envs/Template_Project/checkpoints"

In [169]:
sample = next(iter(test_dl))
imgs, targets = sample
# ! teasdfas


In [11]:
best_hist = get_specific_history(Path(setup_config["c_c_ckp_path"]), "valid_loss", "parquet", "all")
plot_history(best_hist[-1])

,valid_acc,valid_loss,train_loss
epochs,,,
1,0.903326,0.654299,1.256912
2,0.936999,0.364983,0.475674
3,0.949616,0.259002,0.300677
4,0.958138,0.202675,0.223325
5,0.963486,0.167272,0.178494
6,0.967079,0.142815,0.148886
7,0.971257,0.125010,0.127814
8,0.973763,0.111532,0.112072
9,0.976103,0.101109,0.099872
